# Process Checks

In [195]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

## Tickerplant code

In [196]:
from qpython.qconnection import QConnection as qcon
from qpython.qcollection import QDictionary as qdict
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

#set qcon variables
import csv
with open('credentials.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Credentials required are {", ".join(row)}')
            line_count += 1
        else:
            host=row[0]
            un=row[1]
            pswd=row[2]
            print(f'\t host:{row[0]} username: {row[1]} password: {row[2]}.')
            line_count += 1
    print(f'Processed {line_count} lines.')      
                  

Credentials required are host, username, password
	 host:localhost username: admin password: admin.
Processed 2 lines.


In [197]:
#run torq summary
torq_sum= ! "$(cd "$(dirname "${BASH_SOURCE[0]}")" && pwd)"/torq.sh summary

#convert array to numpy array
summary=np.asarray(torq_sum)

#split each element of array by | character
sum_list = [i.split('|') for i in summary]
df = pd.DataFrame(sum_list)

# take first row and use as header for df
new_header = df.iloc[0]
df = df[1:]
df.columns = new_header

#trim whitespace from headers
cols=[]
for i in df.columns:
    cols.append(str.strip(i))
df.columns=cols

#trim whitespace from all objects within dataframe
data = df.select_dtypes(['object'])
df[data.columns] = df.apply(lambda x: x.str.strip())

# function to extract the port number for each process - takes a string argument
def find_portno(process):
    process_info = df.loc[df['PROCESS'] == process]
    PORT = process_info['PORT'].astype(str).astype(int)
    return PORT

#function to decode bytes to strings
def byte_decode(table,cols):
    table[cols] = table[cols].applymap(lambda x: x.decode('utf-8'))

### Process Summary

Table showing process name, status, PID and Port number.
* Process status indicated by colours green (up) and red (down).
* Killtick, tpreplay1 and compression1 should usually have a down status indicated.

In [198]:
# set colour on down processes to red and up processes to green
def colour_down_red(col):
    color = 'red' if 'down' in col else 'green'
    return 'color: %s' % color
df.style.applymap(colour_down_red, subset=['STATUS'])

,TIME,PROCESS,STATUS,PID,PORT
1,15:25:47,discovery1,up,2467,1701
2,15:25:47,tickerplant1,up,2560,1700
3,15:25:47,rdb1,up,2650,1702
4,15:25:47,hdb1,up,2742,1703
5,15:25:47,hdb2,up,2834,1704
6,15:25:47,wdb1,up,2926,1705
7,15:25:47,sort1,up,3020,1706
8,15:25:47,gateway1,up,3112,1707
9,15:25:47,killtick,down,,None
10,15:25:47,monitor1,up,3204,1709


### Count of tables in Tickerplant 

Table to show the count in each table found in the Tickerplant 
* The counts in each of these tables should be 0, as the Tickerplant should not be storing any data.
* If the counts in any of these tables is not 0, this could indicate a slow subscriber.

In [199]:
#table to show tables in TP and the counts of each of them
##counts should all be zero
with qcon(host, port=find_portno('tickerplant1'), username=un, password=pswd,timeout=3.0) as q:
    tablecounts = q("enlist tables[]!count each value each tables[]", pandas=True)
tablecounts

,logmsg,quote,quote_iex,trade,trade_iex
0,0,0,0,0,0


### Tickerplant Log file size increasing

Checks if log messages in the log file of the tickerplant is increasing.
   * If log messages are increasing, the tickerplant is receiving data.
   * If log messages are not increasing, the tickerplant may not be recieving data. 


In [200]:
#log files are increasing over time
with qcon(host, port=find_portno('tickerplant1'), username=un, password=pswd,timeout=3.0) as q:
    log1=(q("hcount .u.L"))
    time.sleep(2)
    log2=(q("hcount .u.L"))
print ("Log file sizes are increasing: ", log1<log2)

Log file sizes are increasing:  True


### Process handles connected to Tickerplant

Table to show the handles of processes connected to the tickerplant and if there are any slow subscribers.
  *  A process may be a slow subscriber if there is a number value in the output queue.

In [201]:
# shows IPC handles with number of bytes waiting in their output queues and what processes are connected
##shows any slow subscribers 
with qcon(host, port=find_portno('tickerplant1'), username=un, password=pswd,timeout=3.0) as q:
    zW=q(".z.W[]", pandas=True)
    hprocesses=q("asc select w,u from .clients.clients", pandas=True)
    byte_decode(hprocesses, ['u'])
# assign columns with new names
keys=pd.DataFrame(zW.keys, columns=['handles'])
values=pd.DataFrame(zW.values, columns=['output queue'])
hprocesses=hprocesses.rename(columns={'u':'processes'})
# apply new names
zW2=keys.join(values)
# join zW2 and hprocesses
zW2=zW2.join(hprocesses.processes)
zW2.set_index('handles', inplace=True)
zW2

,output queue,processes
handles,,
6,NaN,rdb
7,NaN,wdb
8,NaN,feed
9,NaN,chainedtp
10,NaN,metrics
11,NaN,iexfeed
12,NaN,admin


## RDB code

In [202]:
with qcon(host, port=find_portno('rdb1'), username=un, password=pswd,timeout=3.0) as q:
    
    #Check tables in rdb are same as tables in tickerplant 
    tables = q('all 1_tables[] in ((exec w from .servers.SERVERS where proctype=`tickerplant)0)("tables[]")')
    #Check count of tables in rdb - data is being sent from the tickerplant 
    tptordb = q('enlist tables[]!count each value each tables[]', pandas=True)
    #Check rdb can be queried
    rdbtquery = q('-5#select from trade', pandas=True)
    rdbqquery= q('-5#select from quote', pandas=True)
    
    byte_decode(rdbtquery,['sym'])
    byte_decode(rdbqquery,['sym'])
    

    #Check that only data from today is present in the rdb tables
    onedatet = q('select Currentdate:all .z.d=distinct (`date$time) from trade', pandas=True)
    #ondedateq = q('select Currentdate:all .z.d=distinct (`date$time) from quote', pandas=True)

### RDB tables

Checks if the tables in the rdb are the same as the tables in the Tickerplant.

In [203]:
print ("RDB tables are same as Tickerplant tables : ", tables)

RDB tables are same as Tickerplant tables :  True


### RDB table counts

Checks to see if data is being sent from the Tickerplant to the RDB
* Counts for heartbeat and logmsg should be 0
* If counts for the other tables are 0, the RDB has been sent no data for today

In [204]:
tptordb.set_index(tptordb.columns.tolist())

,,,,,
heartbeat,logmsg,quote,quote_iex,trade,trade_iex
0,0,1022223,0,214366,0


### RDB Query
Checks to see if tables in the RDB can be queried 

#### Trade table query

In [205]:
rdbtquery

,time,sym,price,size,stop,cond,ex
0,2019-10-25 14:25:50.400557,DELL,16.47,52,False,T,N
1,2019-10-25 14:25:50.400557,MSFT,40.03,18,False,A,N
2,2019-10-25 14:25:50.400557,AMD,39.55,88,False,8,N
3,2019-10-25 14:25:50.400557,HPQ,45.47,16,False,A,O
4,2019-10-25 14:25:50.400557,HPQ,45.51,81,False,C,O


#### Quote table query

In [206]:
rdbqquery

,time,sym,bid,ask,bsize,asize,mode,ex
0,2019-10-25 14:25:51.000011,GOOG,64.27,65.81,33,56,L,N
1,2019-10-25 14:25:51.000011,DOW,15.13,15.68,84,88,L,O
2,2019-10-25 14:25:51.000011,MSFT,39.92,40.08,40,23,A,N
3,2019-10-25 14:25:51.000011,AMD,39.18,40.48,29,47,O,N
4,2019-10-25 14:25:51.000011,IBM,81.83,83.17,80,25,Y,N


### RDB Date Checks
Check to see if date in RDB tables is the current date

In [207]:
def color_true(val):
    color  ='pink' if val==False else 'green'
    return 'background-color: %s' %color
onedatet.style.applymap(color_true)

,Currentdate
0,True


### HDB Code

In [211]:
with qcon(host, port=find_portno('hdb1'), username=un, password=pswd,timeout=3.0) as q:
    
    #Check hdb table counts excl. eod_summary and eod_summary_iex
    hdbtablecount=q('enlist tables[]!count each value each tables[]',pandas=True)

    
    lastdayquery=q('5#select from trade where date=.z.d-1', pandas=True)
    byte_decode(lastdayquery,['sym'])
    
    lastqquery=q('5#select from quote where date=.z.d-1', pandas=True)
    byte_decode(lastqquery,['sym'])


### HDB Table Counts
* Counts for heartbeat and logmsg should be 0
* If counts for other tables are 0, no data has been recieved for the day before

In [ ]:
hdbtablecount.set_index(hdbtablecount.columns.tolist())

### HDB Query
Check to see if HDB tables can be queried

#### Trade table query 

In [ ]:
lastdayquery

#### Quote table query

In [212]:
lastqquery

,date,time,sym,bid,ask,bsize,asize,mode,ex
0,2019-10-24,2019-10-24 07:56:05.464791,MSFT,37.34,38.47,61,66,O,N
1,2019-10-24,2019-10-24 07:56:05.464791,MSFT,37.69,38.40,18,43,B,N
2,2019-10-24,2019-10-24 07:56:05.464791,MSFT,37.28,39.14,81,98,Y,N
3,2019-10-24,2019-10-24 07:56:05.464791,MSFT,38.02,38.60,57,15,N,N
4,2019-10-24,2019-10-24 07:56:05.774666,MSFT,37.58,38.45,22,60,I,N


### Process Memory Usage

Table to show memory usage of each process

In [213]:
pid=!ps -o pid,user,%mem,%cpu,command ax |grep stackid  #process status commmand for TorQ processes
df2=np.asarray(pid)                                     #make ps ouput into an array
df3=[i.split(' ') for i in df2]                         #split array by whitespace
df4=pd.DataFrame(df3)                                   #make array into a pandas dataframe
df4=df4.drop([0,1,4,6,8,9,10,11,12,13,14,16,17,18,19,20,21,22,23,24,25,26,27,28,29],1) #drop unwanted/empty columns
df4=df4.drop([17,18],0)                                 #drop last two rows of non-TorQ related process status info
df4.columns=('PID','USER','%MEMORY','%CPU','process')   #rename columns appropriately
df4

,PID,USER,%MEMORY,%CPU,process
0,2467,nwatter+,0.2,0.1,discovery1
1,2560,nwatter+,0.0,0.1,tickerplant1
2,2650,nwatter+,5.7,0.1,rdb1
3,2742,nwatter+,0.5,0.0,hdb1
4,2834,nwatter+,0.2,0.0,hdb2
5,2926,nwatter+,0.2,0.1,wdb1
6,3020,nwatter+,0.2,0.1,sort1
7,3112,nwatter+,0.3,0.1,gateway1
8,3204,nwatter+,0.3,0.1,monitor1
9,3296,nwatter+,0.2,0.0,housekeeping1
